In [10]:
# External package imports
import tensorflow as tf
import glob
import os
import numpy as np
import matplotlib.pyplot as plt

In [11]:
# Import the custom data loader
from data_loader import initialize_dataset

In [12]:
# Get a list of the training record files
tfrecord_files = glob.glob(os.path.join("data", "train_tfrecords", "*.tfrec"))

# Define the number of files for training, 70% of files
n_file_train = int(0.7 * len(tfrecord_files))

# Split the record files into 70% training and 30% validation
tfrecord_files_train = tfrecord_files[:n_file_train]
tfrecord_files_valid = tfrecord_files[n_file_train:]

In [13]:
# Define batch size
BATCH_SIZE = 256

# Define processed image size
IMG_SIZE = (96, 96)

In [14]:
# Initialize the training data loader, with random flipping and rotations
dataset_train = initialize_dataset(tfrecord_files_train, batch_size=BATCH_SIZE, img_size=IMG_SIZE, flip=True, rot=(-1, 1))

# Initialize the validation loader, with no flipping or rotations
dataset_valid = initialize_dataset(tfrecord_files_valid, img_size=IMG_SIZE)

In [15]:
# Define our baseline model to test the loader
model = tf.keras.Sequential(layers=[
    tf.keras.Input(shape=(*IMG_SIZE, 3)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', input_shape=(*IMG_SIZE, 3)),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(200, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(5, activation="softmax", kernel_initializer="he_normal")
])

# Compile the model
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    metrics=["accuracy"]
)

# View a summary of the compiled model
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 96, 96, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 48, 48, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 36864)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               7373000   
_________________________________________________________________
dense_3 (Dense)              (None, 5)                

In [16]:
# Run the baseline model for a few epochs to test the loader
train_hist = model.fit(
    x=dataset_train,
    epochs=3,
    validation_data=dataset_valid
)

Epoch 1/3
58/58 [==============================] - 124s 2s/step - loss: 12.7143 - accuracy: 0.4397 - val_loss: 3.1904 - val_accuracy: 0.5302
Epoch 2/3
58/58 [==============================] - 124s 2s/step - loss: 2.4013 - accuracy: 0.4752 - val_loss: 2.5738 - val_accuracy: 0.5770
Epoch 3/3
58/58 [==============================] - 124s 2s/step - loss: 2.1357 - accuracy: 0.4880 - val_loss: 2.1603 - val_accuracy: 0.5616
